In [3]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import tensorflow as tf

In [4]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

interpreter = tf.lite.Interpreter(model_path='C:\\Users\\Santosh\\Desktop\\fyp\\src\\saved_models\\dynamic.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

ideal_angles = {
    'downdog': {
        'right_elbow': {'angle': 180, 'range': 15},
        'left_elbow': {'angle': 180, 'range': 15},
        'right_shoulder': {'angle': 180, 'range': 15},
        'left_shoulder': {'angle': 180, 'range': 15},
        'right_hip': {'angle': 90, 'range': 10},
        'left_hip': {'angle': 90, 'range': 10},
    },
    'goddess': {
        'right_knee': {'angle': 120, 'range': 15},
        'left_knee': {'angle': 120, 'range': 15},
        'right_hip': {'angle': 120, 'range': 15},
        'left_hip': {'angle': 120, 'range': 15},
    },
    'plank': {
        'right_elbow': {'angle': 180, 'range': 10},
        'left_elbow': {'angle': 180, 'range': 10},
        'right_shoulder': {'angle': 180, 'range': 15},
        'left_shoulder': {'angle': 180, 'range': 15},
        'right_hip': {'angle': 180, 'range': 10},
        'left_hip': {'angle': 180, 'range': 10},
        'right_knee': {'angle': 180, 'range': 10},
        'left_knee': {'angle': 180, 'range': 10},
    },
    'tree': {
        'right_knee': {'angle': 180, 'range': 10},
        'left_knee': {'angle': 90, 'range': 15},
        'right_hip': {'angle': 180, 'range': 10},
        'left_hip': {'angle': 150, 'range': 15},
        'right_ankle': {'angle': 90, 'range': 10},
        'left_ankle': {'angle': 90, 'range': 10},
    },
    'warrior': {
        'right_knee': {'angle': 90, 'range': 15},
        'left_knee': {'angle': 180, 'range': 10},
        'right_hip': {'angle': 120, 'range': 15},
        'left_hip': {'angle': 180, 'range': 10},
        'right_ankle': {'angle': 90, 'range': 10},
        'left_ankle': {'angle': 90, 'range': 10},
        'right_shoulder': {'angle': 90, 'range': 10},
        'left_shoulder': {'angle': 90, 'range': 10},
    }
}

def preprocess_image(image, target_size=(75, 75)):
    img = cv2.resize(image, target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array.astype(np.float32)  # TFLite often requires float32
    return img_array

def predict(preprocessed_image, interpreter):
    pose_map = {0: 'downdog', 1: 'goddess', 2: 'plank', 3: 'tree', 4: 'warrior'}
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], preprocessed_image)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get the confidence score of the highest prediction
    confidence = np.max(output_data)
    predicted_class_index = np.argmax(output_data, axis=1)[0]
    
    # Set a confidence threshold (adjust this value based on your needs)
    CONFIDENCE_THRESHOLD = 0.7
    
    if confidence < CONFIDENCE_THRESHOLD:
        return 'unknown'
    
    predicted_pose_name = pose_map[predicted_class_index]
    return predicted_pose_name

def calculate_angle(a, b, c):
    # Function to calculate the angle between three points
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle

def get_pose_angles(landmarks):
    # Define landmark positions
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    # Calculate angles for various joints
    return {
        # Elbow angles
        'right_elbow': calculate_angle(right_shoulder, right_elbow, right_wrist),
        'left_elbow': calculate_angle(left_shoulder, left_elbow, left_wrist),
        
        # Knee angles
        'right_knee': calculate_angle(right_hip, right_knee, right_ankle),
        'left_knee': calculate_angle(left_hip, left_knee, left_ankle),
        
        # Shoulder angles (relative to torso)
        'right_shoulder': calculate_angle(right_elbow, right_shoulder, right_hip),
        'left_shoulder': calculate_angle(left_elbow, left_shoulder, left_hip),
        
        # Hip angles (relative to torso)
        'right_hip': calculate_angle(right_shoulder, right_hip, right_knee),
        'left_hip': calculate_angle(left_shoulder, left_hip, left_knee),
        
        # Ankle angles (relative to lower leg)
        'right_ankle': calculate_angle(right_knee, right_ankle, 
                                       [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]),
        'left_ankle': calculate_angle(left_knee, left_ankle, 
                                      [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]),
        
        # Additional angles for more detailed analysis
        # Torso angle (vertical)
        'torso_vertical': calculate_angle(
            [(right_shoulder[0] + left_shoulder[0]) / 2, 0],  # Point directly above midpoint of shoulders
            [(right_shoulder[0] + left_shoulder[0]) / 2, (right_shoulder[1] + left_shoulder[1]) / 2],  # Midpoint of shoulders
            [(right_hip[0] + left_hip[0]) / 2, (right_hip[1] + left_hip[1]) / 2]  # Midpoint of hips
        ),
        
        # Arm angles (relative to torso)
        'right_arm': calculate_angle(right_shoulder, right_elbow, right_hip),
        'left_arm': calculate_angle(left_shoulder, left_elbow, left_hip),
        
        # Leg angles (relative to torso)
        'right_leg': calculate_angle(right_hip, right_knee, right_shoulder),
        'left_leg': calculate_angle(left_hip, left_knee, left_shoulder),
    }

def compare_angles(detected_angles, ideal_angles):
    feedback = []
    for joint, angle_info in ideal_angles.items():
        if joint in detected_angles:
            detected = detected_angles[joint]
            ideal = angle_info['angle']
            acceptable_range = angle_info['range']
            
            diff = detected - ideal
            if abs(diff) > acceptable_range:
                if diff > 0:
                    feedback.append(f"Bend your {joint} less (current: {detected:.1f}°, ideal: {ideal}°)")
                else:
                    feedback.append(f"Bend your {joint} more (current: {detected:.1f}°, ideal: {ideal}°)")
    return feedback

def get_overall_pose_score(detected_angles, ideal_angles):
    total_score = 0
    num_joints = len(ideal_angles)
    
    for joint, angle_info in ideal_angles.items():
        if joint in detected_angles:
            detected = detected_angles[joint]
            ideal = angle_info['angle']
            acceptable_range = angle_info['range']
            
            diff = abs(detected - ideal)
            if diff <= acceptable_range:
                total_score += 1
            else:
                total_score += max(0, 1 - (diff - acceptable_range) / (180 - acceptable_range))
    
    return (total_score / num_joints) * 100

In [5]:
class AngleBuffer:
    def __init__(self, window_size=10):
        self.window_size = window_size
        self.angles_buffer = {}
    
    def update(self, angles_dict):
        for joint, angle in angles_dict.items():
            if joint not in self.angles_buffer:
                self.angles_buffer[joint] = []
            self.angles_buffer[joint].append(angle)
            if len(self.angles_buffer[joint]) > self.window_size:
                self.angles_buffer[joint].pop(0)
    
    def get_averaged_angles(self):
        averaged_angles = {}
        for joint, angles in self.angles_buffer.items():
            averaged_angles[joint] = sum(angles) / len(angles)
        return averaged_angles

In [6]:
def real_time_pose_estimation():
    # Initialize the Pose Estimation model
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
    
    # Initialize the angle buffer
    angle_buffer = AngleBuffer(window_size=10)
    
    # Initialize pose buffer for stable pose prediction
    pose_buffer = []
    POSE_BUFFER_SIZE = 5

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            # Get current frame angles
            detected_angles = get_pose_angles(results.pose_landmarks.landmark)
            
            # Update the angle buffer
            angle_buffer.update(detected_angles)
            
            # Get smoothed angles
            smoothed_angles = angle_buffer.get_averaged_angles()

            # Pose classification
            skeleton_image = np.zeros_like(frame)
            mp_drawing.draw_landmarks(
                skeleton_image, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=5, circle_radius=5),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
            )
            preprocessed_image = preprocess_image(skeleton_image)
            predicted_pose = predict(preprocessed_image, interpreter)
            
            # Update pose buffer
            pose_buffer.append(predicted_pose)
            if len(pose_buffer) > POSE_BUFFER_SIZE:
                pose_buffer.pop(0)
            
            # Get most common pose in buffer
            stable_pose = max(set(pose_buffer), key=pose_buffer.count)

            if stable_pose != 'unknown':
                # Use smoothed angles for feedback and scoring
                feedback = compare_angles(smoothed_angles, ideal_angles[stable_pose])
                pose_score = get_overall_pose_score(smoothed_angles, ideal_angles[stable_pose])
                
                # Display feedback
                cv2.putText(frame, f"Predicted Pose: {stable_pose}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.putText(frame, f"Pose Score: {pose_score:.2f}%", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                for i, corr in enumerate(feedback):
                    cv2.putText(frame, corr, (10, 90 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                # Display for unknown poses
                cv2.putText(frame, "Predicted Pose: Unknown", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                # Display smoothed angles
                y_pos = 60
                for joint, angle in smoothed_angles.items():
                    cv2.putText(frame, f"{joint}: {angle:.1f}°", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    y_pos += 30
                    if y_pos > frame.shape[0] - 30:
                        break

        cv2.imshow('Pose Estimation', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
if __name__ == "__main__":
    real_time_pose_estimation()